### importing require packages

In [46]:
from __future__ import print_function

import json
import os
import numpy as np
import sys
import h5py

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from embeddings import Embeddings
from keras.callbacks import ModelCheckpoint

from nltk.tokenize import word_tokenize
import keras.backend as k
import tensorflow as tf

In [53]:
session = tf.Session()

## Instantiate Embeddings 

In [22]:
embeddings = Embeddings(100, 3, 1, 4)

### getting data from preprocessing

In [23]:
word2vec_weights = embeddings.get_weights()
word2index, index2word = embeddings.get_vocabulary()
word2vec_model = embeddings.get_model()
tokenized_indexed_sentences = embeddings.get_tokenized_indexed_sentences()

### generating training data

In [24]:
window_size = 5
vocab_size = len(word2index)
print(vocab_size)
#sorted(window_size,reverse=True)
#sentence_max_length = max([len(sentence) for sentence in tokenized_indexed_sentence ])

138607


In [25]:
seq_in = []
seq_out = []
# generating dataset
for sentence in tokenized_indexed_sentences:
    for i in range(len(sentence)-window_size-1):
        x = sentence[i:i + window_size]
        y = sentence[i + window_size]
        seq_in.append(x)#[]
        seq_out.append(word2vec_weights[y])

# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

Number of samples :  3293924


## Defining model

In [68]:
def ACU(ytrue, ypred):
    print(tf.eval(ytrue))
    print(tf.eval(ytrue))
    print(session.run(ytrue))
    return k.mean(ytrue)

In [69]:
# Changes to the model to be done here
model = Sequential()
model.add(Embedding(input_dim=word2vec_weights.shape[0], output_dim=word2vec_weights.shape[1], weights=[word2vec_weights]))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(word2vec_weights.shape[1], activation='sigmoid'))
model.load_weights("../weights/Model-LSTM-2-Layers-512-50-Epochs/weights.09-0.20.hdf5")
model.compile(loss=ACU, optimizer='adam',metrics=['accuracy'])
model.summary()

AttributeError: module 'tensorflow' has no attribute 'eval'

In [ ]:
model_weights_path = "../weights/Model-LSTM-2-Layers-512-50-Epochs"
if not os.path.exists(model_weights_path):
    os.makedirs(model_weights_path)
checkpoint_path = model_weights_path + '/weights-02.{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

## Train Model

In [30]:
model.fit(seq_in, seq_out, epochs=1, verbose=1, validation_split=0.2, batch_size=256, callbacks=[checkpoint])

Train on 2635139 samples, validate on 658785 samples
Epoch 1/1
   4864/2635139 [..............................] - ETA: 6058s - loss: 0.8101 - acc: 0.1910 - ACU: -0.0567

KeyboardInterrupt: 

### model predict

In [9]:
start = 97
pattern = list(seq_in[start])
print("\"",' '.join(index2word[index] for index in pattern))
for i in range(100):
    prediction = model.predict(np.array([pattern]))
    pred_word = word2vec_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word + " ")
    pattern.append(word2index[pred_word])
    pattern = pattern[1:len(pattern)]

" so that the logo could
solution restricted to restricted restricted corruption problematic in restricted environment strangled by restricted restricted sexes restricted restricted restricted restricted to restricted corruption problematic in restricted environment strangled by restricted restricted sexes restricted restricted restricted restricted to restricted corruption problematic in restricted environment strangled by restricted restricted sexes restricted restricted restricted restricted to restricted corruption problematic in restricted environment strangled by restricted restricted sexes restricted restricted restricted restricted to restricted corruption problematic in restricted environment strangled by restricted restricted sexes restricted restricted restricted restricted to restricted corruption problematic in restricted environment strangled by restricted restricted sexes restricted restricted restricted restricted to 

In [10]:
pred_words = word2vec_model.similar_by_vector(prediction[0])
pred_words

[('to', 0.6187251210212708),
 ('able', 0.49232569336891174),
 ('forced', 0.48312145471572876),
 ('order', 0.46920308470726013),
 ('expected', 0.45870184898376465),
 ('transportations', 0.451442688703537),
 ('reassessed', 0.4452168047428131),
 ('help', 0.4443209171295166),
 ('trying', 0.43627357482910156),
 ('attempting', 0.43250030279159546)]

In [11]:
from itertools import product

In [12]:
word_pairs = list(product([word2vec_model.similar_by_vector(seq_out[97])[0][0]], list(zip(*pred_words))[0]))

In [44]:
max([word2vec_model.similarity(w1, w2) for w1, w2 in word_pairs])

0.36474725199852309